In [7]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime

sns.set_style('whitegrid')

import multiprocessing
cpu_cores = multiprocessing.cpu_count()

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [8]:
uber = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/other_datasets//uber_data2.csv').drop('Unnamed: 0', 1)
uber.head()

,Year,Month,Day,Hour,1,2,3,4,5,6,...,264,265,Dew PointF,Gust SpeedMPH,Humidity,Sea Level PressureIn,TemperatureF,VisibilityMPH,Wind SpeedMPH,WindDirDegrees
0,2015,2,1,0,0.0,0.0,4.0,39.0,0.0,0.0,...,0.0,0.0,6.1,0.0,55.0,30.19,19.9,10.0,0.0,0
1,2015,2,1,1,0.0,0.0,0.0,58.0,0.0,0.0,...,0.0,0.0,6.1,0.0,53.0,30.19,21.0,10.0,3.5,250
2,2015,2,1,2,0.0,0.0,0.0,30.0,0.0,0.0,...,0.0,0.0,6.1,0.0,51.0,30.19,21.9,10.0,3.5,0
3,2015,2,1,3,0.0,0.0,0.0,23.0,0.0,0.0,...,0.0,1.0,7.0,0.0,50.0,30.19,23.0,10.0,3.5,0
4,2015,2,1,4,0.0,0.0,0.0,11.0,0.0,0.0,...,0.0,0.0,8.1,0.0,51.0,30.19,24.1,10.0,0.0,0


In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
#Scale data between 0,1 for LSTM/GRU
MMS = MinMaxScaler()
uber = pd.DataFrame(MMS.fit_transform(uber), columns=uber.columns)

#  Remember to shift the data so we are predicting one time step future
X = uber.iloc[:-1,:]
y = uber.iloc[1:,4:269] #265 locationIDs
#y = uber.ix[:, 4:] #Non_Weather Set

# Train-Test-Split-80/20
X_train, X_test = X[:int(round(len(X)*0.8, 0))], X[int(round(len(X)*0.8, 0)):]
y_train, y_test = y[:int(round(len(y)*0.8, 0))], y[int(round(len(y)*0.8, 0)):]

#Check data validity
print len(X_train) + len(X_test), len(y_train) + len(y_test)

3598 3598


In [11]:
from sklearn.ensemble import RandomForestRegressor

In [19]:
start_time = time.time()
print 'Executing ||', str(datetime.now())
RF = RandomForestRegressor(n_estimators=250, max_depth=None, n_jobs=cpu_cores, verbose=0).fit(X_train, y_train)

print 'train_score =', RF.score(X_train, y_train)
print 'test_score =', RF.score(X_test, y_test)

print 'Runtime =', round((time.time() - start_time), 2), 'seconds'

Executing || 2017-01-21 16:10:43.651107
train_score = 0.928456067169
test_score = 0.415518465682
Runtime = 257.75 seconds
